# TMP notebook
competition : RSNA Screening Mammography Breast Cancer Detection  
url : https://www.kaggle.com/competitions/rsna-breast-cancer-detection

# 2023/1/22
テーブルデータのみのlightgbmでどれくらい精度出るか検証  
* 交差検証なし、パラメータチューニングなし

## 結果
* LB - 0.04

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter('ignore')

def pre_view(df):
    if 'view' in df.columns.tolist():
        df['view'] = df['view'].apply(lambda x: x if x=='CC' or x=='MLO' else 'others')
    else:
        pass
    return df

train = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')
test = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')
submit = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/sample_submission.csv')

column = ['laterality', 'view', 'age', 'implant', 'cancer']
new_train = train[column]
new_train = new_train.dropna()
new_train = pre_view(new_train)
new_train_dum = pd.get_dummies(new_train)
train_cancer = new_train_dum[new_train_dum.cancer==1]
train_no_cancer = new_train_dum[new_train_dum.cancer==0]
tmp = train_no_cancer.sample(n=1158, random_state=0)
concat_train = pd.concat([train_cancer, tmp])

concat_train = concat_train.reset_index()
X = concat_train.drop(columns=['cancer'])
y = concat_train[['cancer']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)
# X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=0, stratify=y_test)

model = lgb.LGBMClassifier(random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print(f1_score(y_test, pred))

tmp = test[['laterality', 'view', 'age', 'implant']]
tmp = pre_view(tmp)
tmp_ = pd.concat([new_train, tmp])
tmp_dum = pd.get_dummies(tmp_)
tmp_dum = tmp_dum.reset_index()
test_X = tmp_dum.iloc[new_train.shape[0]:]
test_X = test_X.drop(columns=['cancer'])
test_pred = model.predict(test_X)
print(test_pred)
test_copy = test.copy()
test_copy['pred'] = test_pred
tmp = test_copy.groupby('prediction_id')['pred'].mean()
sub = pd.DataFrame(data={'prediction_id': tmp.index.tolist(), 'cancer': tmp.values.tolist()})
display(sub)
sub.to_csv('submission.csv', index=None)

0.6547945205479452
[1 1 1 1]


,prediction_id,cancer
0,10008_L,1.0
1,10008_R,1.0
